In [ ]:
import os
import json
from glob import glob

"""
Due to the error while evaluating multiple validation sets. We have to 
unify all val.json files into a single file.
"""

# Root directory where your data folders are stored
root_dir = 'data'
side = "left"
phase = "val"

# Find all val.json files in the given folder structure
val_files = glob(os.path.join(root_dir, '*/*/image/{}/annotations/{}.json'.format(side, phase)))

# Initialize the combined data structure
combined_data = {
    'images': [],
    'annotations': [],
    'categories': [],
}

# Counters to avoid duplicate ids
image_id_offset = 0
annotation_id_offset = 0

# Loop over all val.json files and merge them
side = "left"
for file in val_files:
    folders = file.split("/")
    date, date_time = folders[1], folders[2]
    img_rel_path = os.path.join(date, date_time, "image", side, phase)
    
    with open(file, 'r') as f:
        data = json.load(f)
    
    # Adjust image_ids and annotation_ids to avoid conflicts
    for image in data['images']:
        # Update file_name to be a relative path from the data folder
        image_path = os.path.relpath(os.path.join(os.path.dirname(file), '..', '..', image['file_name']), root_dir)
        image['file_name'] = os.path.join(img_rel_path, image["file_name"])
        
        # Update image ID
        image['id'] += image_id_offset
    
    for annotation in data['annotations']:
        annotation['id'] += annotation_id_offset
        annotation['image_id'] += image_id_offset

    # Update the combined data
    combined_data['images'].extend(data['images'])
    combined_data['annotations'].extend(data['annotations'])
    
    # Assuming categories are the same across all files
    if not combined_data['categories']:
        combined_data['categories'] = data['categories']
    
    # Update the offsets
    image_id_offset = max(image['id'] for image in combined_data['images']) + 1
    annotation_id_offset = max(annotation['id'] for annotation in combined_data['annotations']) + 1

# Save the combined val.json
output_file = os.path.join(root_dir, 'unified_val.json')
with open(output_file, 'w') as f:
    json.dump(combined_data, f)

print(f"Unified val.json saved to {output_file}")